In [ ]:

import os
import datetime


import pandas as pd
import numpy as np


import matplotlib.pyplot as plt
import seaborn as sns
#import plotly.express as px 
%matplotlib inline


import warnings
warnings.filterwarnings('ignore')

#Progreebar
from tqdm import tqdm

# For transformations and predictions
from scipy.optimize import curve_fit
from yellowbrick.target import FeatureCorrelation
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.cluster import KMeans 
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor

# For scoring
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import r2_score,mean_absolute_error

# For validation
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv('/content/spotify_data.csv')

In [ ]:
df.head()

,valence,year,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo
0,0.0594,1921,0.982,"['Sergei Rachmaninoff', 'James Levine', 'Berli...",0.279,831667,0.211,0,4BJqT0PrAfrxzMOxytFOIz,0.878000,10,0.665,-20.096,1,"Piano Concerto No. 3 in D Minor, Op. 30: III. ...",4,1921,0.0366,80.954
1,0.9630,1921,0.732,['Dennis Day'],0.819,180533,0.341,0,7xPhfUan2yNtyFG0cUWkt8,0.000000,7,0.160,-12.441,1,Clancy Lowered the Boom,5,1921,0.4150,60.936
2,0.0394,1921,0.961,['KHP Kridhamardawa Karaton Ngayogyakarta Hadi...,0.328,500062,0.166,0,1o6I8BglA6ylDMrIELygv1,0.913000,3,0.101,-14.850,1,Gati Bali,5,1921,0.0339,110.339
3,0.1650,1921,0.967,['Frank Parker'],0.275,210000,0.309,0,3ftBPsC5vPBKxYSee08FDH,0.000028,5,0.381,-9.316,1,Danny Boy,3,1921,0.0354,100.109
4,0.2530,1921,0.957,['Phil Regan'],0.418,166693,0.193,0,4d6HGyGT8e121BsdKmw9v6,0.000002,3,0.229,-10.096,1,When Irish Eyes Are Smiling,2,1921,0.0380,101.665


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 170653 entries, 0 to 170652
Data columns (total 19 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   valence           170653 non-null  float64
 1   year              170653 non-null  int64  
 2   acousticness      170653 non-null  float64
 3   artists           170653 non-null  object 
 4   danceability      170653 non-null  float64
 5   duration_ms       170653 non-null  int64  
 6   energy            170653 non-null  float64
 7   explicit          170653 non-null  int64  
 8   id                170653 non-null  object 
 9   instrumentalness  170653 non-null  float64
 10  key               170653 non-null  int64  
 11  liveness          170653 non-null  float64
 12  loudness          170653 non-null  float64
 13  mode              170653 non-null  int64  
 14  name              170653 non-null  object 
 15  popularity        170653 non-null  int64  
 16  release_date      17

In [ ]:
df["artists"] = df["artists"].str.replace("[","")
df["artists"] = df["artists"].str.replace("]","")
df["artists"] = df["artists"].str.replace("'","")

In [ ]:
df.head()

,valence,year,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo
0,0.0594,1921,0.982,"Sergei Rachmaninoff, James Levine, Berliner Ph...",0.279,831667,0.211,0,4BJqT0PrAfrxzMOxytFOIz,0.878000,10,0.665,-20.096,1,"Piano Concerto No. 3 in D Minor, Op. 30: III. ...",4,1921,0.0366,80.954
1,0.9630,1921,0.732,Dennis Day,0.819,180533,0.341,0,7xPhfUan2yNtyFG0cUWkt8,0.000000,7,0.160,-12.441,1,Clancy Lowered the Boom,5,1921,0.4150,60.936
2,0.0394,1921,0.961,KHP Kridhamardawa Karaton Ngayogyakarta Hadini...,0.328,500062,0.166,0,1o6I8BglA6ylDMrIELygv1,0.913000,3,0.101,-14.850,1,Gati Bali,5,1921,0.0339,110.339
3,0.1650,1921,0.967,Frank Parker,0.275,210000,0.309,0,3ftBPsC5vPBKxYSee08FDH,0.000028,5,0.381,-9.316,1,Danny Boy,3,1921,0.0354,100.109
4,0.2530,1921,0.957,Phil Regan,0.418,166693,0.193,0,4d6HGyGT8e121BsdKmw9v6,0.000002,3,0.229,-10.096,1,When Irish Eyes Are Smiling,2,1921,0.0380,101.665


In [ ]:
def normalize_column(col):
  max_d = df[col].max()
  min_d = df[col].min()
  df[col] = (df[col]-min_d)/(max_d - min_d)

In [ ]:
num_types = ['int16','int32','int64','float16','float32','float64']
num = df.select_dtypes(include = num_types)

In [ ]:
for col in num.columns:
  normalize_column(col)
  #minmaxscalar()

In [ ]:
#k-means
km = KMeans(n_clusters = 25)
pred = km.fit_predict(num)
df['pred'] = pred
normalize_column('pred')

In [ ]:
df.head()

,valence,year,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo,pred
0,0.0594,0.0,0.985944,"Sergei Rachmaninoff, James Levine, Berliner Ph...",0.282389,0.153112,0.211,0.0,4BJqT0PrAfrxzMOxytFOIz,0.878000,0.909091,0.665,0.624916,1.0,"Piano Concerto No. 3 in D Minor, Op. 30: III. ...",0.04,1921,0.037732,0.332450,0.125000
1,0.9630,0.0,0.734940,Dennis Day,0.828947,0.032496,0.341,0.0,7xPhfUan2yNtyFG0cUWkt8,0.000000,0.636364,0.160,0.744797,1.0,Clancy Lowered the Boom,0.05,1921,0.427835,0.250243,0.625000
2,0.0394,0.0,0.964859,KHP Kridhamardawa Karaton Ngayogyakarta Hadini...,0.331984,0.091685,0.166,0.0,1o6I8BglA6ylDMrIELygv1,0.913000,0.272727,0.101,0.707071,1.0,Gati Bali,0.05,1921,0.034948,0.453125,0.041667
3,0.1650,0.0,0.970884,Frank Parker,0.278340,0.037954,0.309,0.0,3ftBPsC5vPBKxYSee08FDH,0.000028,0.454545,0.381,0.793736,1.0,Danny Boy,0.03,1921,0.036495,0.411113,0.250000
4,0.2530,0.0,0.960843,Phil Regan,0.423077,0.029932,0.193,0.0,4d6HGyGT8e121BsdKmw9v6,0.000002,0.272727,0.229,0.781521,1.0,When Irish Eyes Are Smiling,0.02,1921,0.039175,0.417503,0.083333


In [ ]:
#Song Recommender
class Song_Recommender():
    """
    Neighbourhood Based Collborative Filterng REcoomendation System using similarity Metrics
    Manhattan Distance is calculated for all songs and Recommend Songs that are similar to it based on any given song
    """
    def __init__(self, data):
        self.data_ = data
    
    #function which returns recommendations, we can also choose the amount of songs to be recommended
    def get_recommendations(self, song_name, n_top):
        distances = []
        #choosing the given song_name and dropping it from the data
        song = self.data_[(self.data_.name.str.lower() == song_name.lower())].head(1).values[0]
        rem_data = self.data_[self.data_.name.str.lower() != song_name.lower()]
        for r_song in tqdm(rem_data.values):
            dist = 0
            for col in np.arange(len(rem_data.columns)):
                #indeces of non-numerical columns(id,Release date,name,artists)
                if not col in [3,8,14,16]:
                    #calculating the manhettan distances for each numerical feature
                    dist = dist + np.absolute(float(song[col]) - float(r_song[col]))
            distances.append(dist)
        rem_data['distance'] = distances
        #sorting our data to be ascending by 'distance' feature
        rem_data = rem_data.sort_values('distance')
        columns = ['artists', 'name']
        return rem_data[columns][:n_top]

In [ ]:
recommander = Song_Recommender(df)

In [ ]:
#Gati Bali
recommander.get_recommendations('Gati Mrak Ati',2)

100%|██████████| 170652/170652 [00:10<00:00, 16045.81it/s]


,artists,name
2,KHP Kridhamardawa Karaton Ngayogyakarta Hadini...,Gati Bali
108959,"Frédéric Chopin, William Kapell","Mazurkas, Op. 24: No. 1 in G Minor"
